In [1]:
'''
聚水潭仓库盘存表，与产成品仓库盘存表对比
'''
import os
import datetime
import easygui
import numpy as np
import pandas as pd
import openpyxl
import re


col_lst = ['单据日期','业务类型','单据编号','单据类型','仓库','存货分类编码','存货分类','存货编码','存货','规格型号','入库数量',
 '入库数量2',
 '入库计量单位组合',
 '入库金额',
 '出库数量',
 '出库数量2']

fname = r"F:\a00nutstore\008\zw08\电商\聚水潭\聚水潭商品库存-20241125截止下午3点13分.xlsx"
# msg = '请点选出入库流水'
# fname = easygui.fileopenbox(msg)
path,_ = os.path.split(fname)
os.chdir(path)
df = pd.read_excel(fname) #,dtype= {'Unnamed: 6':str}
df
# df = df.iloc[:, 1:]
# df.columns = col_lst
# df = df[~df['存货编码'].isnull()]
# df1 = df[['存货分类编码','存货编码','存货','规格型号','入库数量2']]




,商品编码,商品名称,实际库存数,可用数,公有可用数
0,1101,总分类账,90,90,90
1,1103,银行存款日记账,84,84,84
2,1105,现金日记账,64,64,64
3,1106,三栏式明细类账,62,62,62
4,1110,库存材料(商品)明细分类账,102,102,102
...,...,...,...,...,...
747,合同,合同,0,0,0
748,介绍信,介绍信,0,0,0
749,启用表（张）,启用表（张）,0,0,0
750,商品卡,商品卡,0,0,0


In [2]:
df1 = df[['商品编码','实际库存数']]
df1.columns = ['code','dianshang']
df1 = df1[df1.dianshang != 0]
df1

,code,dianshang
0,1101,90
1,1103,84
2,1105,64
3,1106,62
4,1110,102
...,...,...
720,XJ401S,250
726,XJ407,260
735,XJ501,410
739,XJ505,370


In [3]:
lst = ['仓库编码',
       '仓库',
       '存货分类 (1级)',
       '存货分类 (2级)',
       '存货分类 (3级)',
       '存货分类 (4级)',
       '存货分类 (5级)',
       '存货编码',
       '存货',
       '存货代码',
       '数量(主单位)',
       '平均单价',
       '金额',
       '数量(辅单位)',
       '计量单位组合',
       '数量(主单位).1',
       '平均单价.1',
       '金额.1',
       '数量(辅单位).1',
       '计量单位组合.1',
       '数量(主单位).2',
       '平均单价.2',
       '金额.2',
       '数量(辅单位).2',
       '计量单位组合.2',
       '数量(主单位).3',
       '平均单价.3',
       '金额.3',
       '辅数量',
       '计量单位组合.3']

lst2 = [
    'store',
    'num',
    'class01',
    'class02',
    'class03',
    'class04',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    '平均单价',
    '金额',
    'begin_jian',
    '计量单位组合',
    'ruku_ben',
    '平均单价.1',
    '金额.1',
    'ruku_jian',
    '计量单位组合.1',
    'chuku_ben',
    '平均单价.2',
    '金额.2',
    'chuku_jian',
    '计量单位组合.2',
    'end_ben',
    '平均单价.3',
    '金额.3',
    'end_jian',
    '计量单位组合.3']

lst3 = [
    'class02',
    'class05',
    'code',
    'stock',
    'content',
    'begin_ben',
    'ruku_ben',
    'chuku_ben',
    'end_ben',
]

lst01_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'begin_jian',
    'ruku_jian',
    'chuku_jian',
]
lst02_yesterday = [
    'class02',
    'class05',
    'code',
    'stock',
    'yesterday_jian',
    'yesterday_ruku',
    'yesterday_chuku',

]
lst_merge = ['class02', 'class05', 'code', 'stock',
             'content',
             'begin_jian', 'yesterday_jian',
             'yesterday_ruku', 'ruku_jian',
             'yesterday_chuku', 'chuku_jian',
             'end_jian']

lst_result0 = ['class05', 'code', 'stock',
               'content',
               'begin_jian', 'yesterday_jian',
               'yesterday_ruku', 'ruku_jian',
               'yesterday_chuku', 'chuku_jian',
               'end_jian']

lst_result = ['类别', '货号', '品名',
              '含量',
              '月初', '上日',
              '本日入库', '入库累计',
              '本日出库', '出库累计',
              '结余']


def chuli(fname):
    df = pd.read_excel(fname, skiprows=7)
    df = df.iloc[:, 1:]
    df.columns = lst
    df.columns = lst2
    df = df.loc[(df.store == '002') | (df.store == '003')]

    df['content'] = df['end_ben'] / df['end_jian']
    df = df[df['store'] != '制表人:']
    df = df[df['store'] != '合计：']
    df = df[df['store'].notnull()]
    df = df.iloc[:, 2:]
    df = df[lst3]
    return df
fname_daku = r"F:\a00nutstore\008\zw08\电商\聚水潭\收发存汇总表002 电商10.26-11.25.xlsx"
df_daku = chuli(fname_daku)
df_daku

,class02,class05,code,stock,content,begin_ben,ruku_ben,chuku_ben,end_ben
0,抄本,材料纸,DB501,单行材料纸,500.000000,470.0,NaN,NaN,470.0
1,抄本,材料纸,FB503,方格材料纸,500.000000,NaN,500.0,33.0,467.0
2,抄本,材料纸,J1601,简装双行材料纸,269.807281,228.0,NaN,102.0,126.0
3,抄本,材料纸,J1602,简装单行材料纸,270.161290,227.0,NaN,160.0,67.0
4,抄本,材料纸,J1603,简装方格材料纸,270.108043,258.0,NaN,33.0,225.0
...,...,...,...,...,...,...,...,...,...
492,账本,普通账芯,1121,16K库存商品明细分类账,60.021436,22.0,60.0,26.0,56.0
493,账本,普通账芯,1123,16K进销存分类账,60.012634,48.0,60.0,13.0,95.0
494,账本,普通账芯,1125,16K存货计数账,60.000000,-4.0,125.0,1.0,120.0
495,账本,装订配件,16KPP账夹,16KPP账夹,259.962049,17.0,260.0,3.0,274.0


In [4]:
df_daku = df_daku[['code','stock','end_ben']]
df_daku = df_daku[~df_daku.end_ben.isnull()]
df_daku = df_daku[df_daku.end_ben != 0]
df_daku

,code,stock,end_ben
0,DB501,单行材料纸,470.0
1,FB503,方格材料纸,467.0
2,J1601,简装双行材料纸,126.0
3,J1602,简装单行材料纸,67.0
4,J1603,简装方格材料纸,225.0
...,...,...,...
492,1121,16K库存商品明细分类账,56.0
493,1123,16K进销存分类账,95.0
494,1125,16K存货计数账,120.0
495,16KPP账夹,16KPP账夹,274.0


In [5]:
df_diff = pd.merge(df1,df_daku,how='outer')
df_diff['dianshang'] = df_diff['dianshang'].fillna(0)
df_diff['end_ben'] = df_diff['end_ben'].fillna(0)
df_diff['diff'] = df_diff.end_ben - df_diff.dianshang
df_diff

,code,dianshang,stock,end_ben,diff
0,1101,90.0,16K总分类账,85.0,-5.0
1,1103,84.0,16K银行存款日记账,77.0,-7.0
2,1105,64.0,16K现金日记账,64.0,0.0
3,1106,62.0,16K三栏式明细分类账,120.0,58.0
4,1110,102.0,16K库存材料(商品)明细分类账,162.0,60.0
...,...,...,...,...,...
526,XJ407,260.0,16K30型精品语文双行缝线本,260.0,0.0
527,XJ501,410.0,32K精品牛卡缝线语文本,410.0,0.0
528,XJ505,370.0,32K精品牛卡缝线练习本,370.0,0.0
529,XJ507,420.0,32K精品牛卡缝线英语本,420.0,0.0


In [6]:
df_diff = df_diff[['code','stock','dianshang','end_ben','diff']]
df_diff.loc['合计','code'] =  ''
df_diff.loc['合计','stock'] =  ''
df_diff.loc['合计','dianshang'] =  df_diff['dianshang'].sum()
df_diff.loc['合计','end_ben'] =  df_diff['end_ben'].sum()
df_diff.loc['合计','diff'] =  df_diff['diff'].sum()
df_diff




C:\Users\redda\AppData\Local\Temp\ipykernel_11480\2527370520.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diff.loc['合计','code'] =  ''


,code,stock,dianshang,end_ben,diff
0,1101,16K总分类账,90.0,85.0,-5.0
1,1103,16K银行存款日记账,84.0,77.0,-7.0
2,1105,16K现金日记账,64.0,64.0,0.0
3,1106,16K三栏式明细分类账,62.0,120.0,58.0
4,1110,16K库存材料(商品)明细分类账,102.0,162.0,60.0
...,...,...,...,...,...
527,XJ501,32K精品牛卡缝线语文本,410.0,410.0,0.0
528,XJ505,32K精品牛卡缝线练习本,370.0,370.0,0.0
529,XJ507,32K精品牛卡缝线英语本,420.0,420.0,0.0
530,电商废次品,电商废次品,0.0,15.0,15.0


In [7]:
df_diff.columns  =  ['存货编码','存货名称','聚水潭','大库','差异']
df_diff.to_excel('聚水潭与大库差异2024-11-25正.xlsx')